> NOMBRE: Elizon Frank Carcausto Mamani

> CODIGO: 170427

# Instalacion e importacion de Recursos

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 37 kB/s 
     |████████████████████████████████| 198 kB 64.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=c475683e0109d8356bd6932a4822d215f19b5f562967f847e9ea808dd2ce325b
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
import re
import math
import nltk
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
sc =SparkContext.getOrCreate()
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Implementacion del Algoritmo TF-IDF

Para la implementacion usaremos los ejemplos del colab adjuntado.

In [3]:
corpus = ["messi messi messi ronaldo ronaldo balon", "messi ronaldo futbol futbol futbol"]
data = [(i+1, corpus[i]) for i in range(len(corpus))]
data

[(1, 'messi messi messi ronaldo ronaldo balon'),
 (2, 'messi ronaldo futbol futbol futbol')]

Creamos el conjunto de datos con el corpus de ejemplo. 

In [4]:
lines=sc.parallelize(data)
lines.collect()

[(1, 'messi messi messi ronaldo ronaldo balon'),
 (2, 'messi ronaldo futbol futbol futbol')]

## CALCULO DEL TF

Para el calculo del TF se usara la formula:
$$tf(t,d) = 1 + log(f_{t,d})$$
Donde:
- **t**: número de veces que aparece la palabra "messi" en el documento
- **d**: número de palabras que tiene el documento

PASO 1: extraemos el nuemro de palabras de cada documento, el (indice + 1)  indica el documento al que pertenece la cantidad.

In [5]:
n_words = [len(i.split()) for i in corpus]

PASO 2: Marcar con 1 la indicencia de cada palabra: ( (IdDoc, Token), 1 ) 











In [6]:
map1=lines.flatMap(lambda x: [ ((x[0],i),1) for i in x[1].split()])
map1.collect()

[((1, 'messi'), 1),
 ((1, 'messi'), 1),
 ((1, 'messi'), 1),
 ((1, 'ronaldo'), 1),
 ((1, 'ronaldo'), 1),
 ((1, 'balon'), 1),
 ((2, 'messi'), 1),
 ((2, 'ronaldo'), 1),
 ((2, 'futbol'), 1),
 ((2, 'futbol'), 1),
 ((2, 'futbol'), 1)]

PASO 3: contar el numero de incidencias de cada token, usando **reduceByKey**: ( ( IdDoc, Token), #incidencias ) 

In [7]:
reduce =map1.reduceByKey(lambda x,y:x+y)
reduce.collect()

[((1, 'ronaldo'), 2),
 ((1, 'balon'), 1),
 ((2, 'messi'), 1),
 ((1, 'messi'), 3),
 ((2, 'ronaldo'), 1),
 ((2, 'futbol'), 3)]

PASO 4:  Calcular el TF usando la formula **Logaritmica** y usando **map**: ( ( Token, ( IdDoc, TF) ) 

In [8]:
tf=reduce.map(lambda x: (x[0][1],(x[0][0], 1 + math.log10(x[1]/n_words[x[0][0]-1])  ) ))
tf.collect()

[('ronaldo', (1, 0.5228787452803375)),
 ('balon', (1, 0.22184874961635637)),
 ('messi', (2, 0.30102999566398125)),
 ('messi', (1, 0.6989700043360187)),
 ('ronaldo', (2, 0.30102999566398125)),
 ('futbol', (2, 0.7781512503836436))]

## CALCULO DE IDF

Para el calculo del IDF se usara la formula:
$$idf({n_t},D) = log(1 + \frac{N}{n_t})$$
Donde:
- **n<sub>t</sub>**: número de documentos en los que aparece la palabra documento
- **D**: número total de documentos en el corpus

PASO 1: Mapeamos los datos con sus TF junto a un contador 1: ( Token, ( IdDoc, TF, 1))

In [9]:
map3=reduce.map(lambda x: (x[0][1],(x[0][0],x[1],1)))
map3.collect()

[('ronaldo', (1, 2, 1)),
 ('balon', (1, 1, 1)),
 ('messi', (2, 1, 1)),
 ('messi', (1, 3, 1)),
 ('ronaldo', (2, 1, 1)),
 ('futbol', (2, 3, 1))]

PASO 2: Extraermos el token y el contador 1. (Token, 1)

In [10]:
map4=map3.map(lambda x:(x[0],x[1][2]))
map4.collect()

[('ronaldo', 1),
 ('balon', 1),
 ('messi', 1),
 ('messi', 1),
 ('ronaldo', 1),
 ('futbol', 1)]

PASO 3: Calculamos el numero de incidencias del token con la funcion **reduceByKey** y un **lamda**. Luego calculamos el valor de IDF haciendo uso de la formula **logaritmica** y del map: (Token, IDF)

In [11]:
reduce2=map4.reduceByKey(lambda x,y:x+y)
idf=reduce2.map(lambda x: (x[0], math.log10(1 + (len(data)/x[1]))))
idf.collect()

[('messi', 0.3010299956639812),
 ('ronaldo', 0.3010299956639812),
 ('balon', 0.47712125471966244),
 ('futbol', 0.47712125471966244)]

## Calculo del TF-IDF

Para el calculo del TF-IDF se usara la formula:
$$tfidf(t,d,D) = tf(t,d) \cdot idf(t,D)$$

Se reutilizara los valores de Tf e IDF ya calculados.

PASO 1: Unimos los dos conjuntos de datos previamente trabajados (TF, IDF). Luego mapeamos cada token con el valor de la multiplicacion de su Tf e IDF. (Token, Tf, IDF, TF-IDF)

In [12]:
rdd=tf.join(idf)
rdd=rdd.map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1],x[1][0][1]*x[1][1]))).sortByKey()
rdd.collect()

[(1, ('ronaldo', 0.5228787452803375, 0.3010299956639812, 0.15740218642452794)),
 (1, ('messi', 0.6989700043360187, 0.3010299956639812, 0.21041093737452465)),
 (1, ('balon', 0.22184874961635637, 0.47712125471966244, 0.10584875377494418)),
 (2,
  ('ronaldo', 0.30102999566398125, 0.3010299956639812, 0.09061905828945656)),
 (2, ('messi', 0.30102999566398125, 0.3010299956639812, 0.09061905828945656)),
 (2, ('futbol', 0.7781512503836436, 0.47712125471966244, 0.37127250094471825))]

PASO 2: Convertirmos en un DataFrame para una mejor visualizacion.

In [13]:
spark = SparkSession(sc)
rdd=rdd.map(lambda x: (x[0],x[1][0],x[1][1],x[1][2],x[1][3]))
hasattr(rdd, "toDF")
rdd.toDF(["DocumentId","Token","TF","IDF","TF-IDF"]).show()

+----------+-------+-------------------+-------------------+-------------------+
|DocumentId|  Token|                 TF|                IDF|             TF-IDF|
+----------+-------+-------------------+-------------------+-------------------+
|         1|ronaldo| 0.5228787452803375| 0.3010299956639812|0.15740218642452794|
|         1|  messi| 0.6989700043360187| 0.3010299956639812|0.21041093737452465|
|         1|  balon|0.22184874961635637|0.47712125471966244|0.10584875377494418|
|         2|ronaldo|0.30102999566398125| 0.3010299956639812|0.09061905828945656|
|         2|  messi|0.30102999566398125| 0.3010299956639812|0.09061905828945656|
|         2| futbol| 0.7781512503836436|0.47712125471966244|0.37127250094471825|
+----------+-------+-------------------+-------------------+-------------------+

